In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch

# Configuration
BASE_PATH = '/kaggle/input/mtcaic3'
OUTPUT_DIR = './preprocessed'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filter design
def design_filters(fs=250.0):
    # Bandpass 1-40 Hz
    bp_b, bp_a = butter(4, [1/(fs/2), 40/(fs/2)], btype='band')
    # Notch at 50 Hz
    notch_b, notch_a = iirnotch(50/(fs/2), Q=30)
    return bp_b, bp_a, notch_b, notch_a

# Preprocessing steps for one trial
def preprocess_trial(df):
    eeg_cols = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
    motion_cols = ['AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3']
    val_col = 'Validation'

    # 1) Motion artifact detection
    motion_mag = np.sqrt((df[motion_cols]**2).sum(axis=1))
    mot_thresh = np.percentile(motion_mag, 95)
    bad_mask = motion_mag > mot_thresh

    # 2) Mask EEG
    data = df[eeg_cols].copy().values
    data[bad_mask, :] = np.nan
    data[df[val_col] == 0, :] = np.nan

    # 3) Interpolation
    for ch in range(data.shape[1]):
        col = data[:, ch]
        nans = np.isnan(col)
        if nans.all():
            continue
        idx = np.arange(len(col))
        data[nans, ch] = np.interp(idx[nans], idx[~nans], col[~nans])

    # 4) Filtering
    bp_b, bp_a, notch_b, notch_a = design_filters()
    for ch in range(data.shape[1]):
        data[:, ch] = filtfilt(bp_b, bp_a, data[:, ch])
        data[:, ch] = filtfilt(notch_b, notch_a, data[:, ch])

    # 5) Baseline correction (first 0.5s)
    bs = int(0.5 * 250)
    baseline = data[:bs].mean(axis=0)
    data -= baseline
    return data

# Load index DataFrame
def load_index(fname, label_col=True):
    df = pd.read_csv(os.path.join(BASE_PATH, fname))
    cols = ['id','subject_id','task','trial_session','trial'] + (['label'] if label_col else [])
    return df[cols]

# Process a split, grouping by task
def process_split(df, has_label=True):
    data_dict = {'MI': {'X': [], 'y': []}, 'SSVEP': {'X': [], 'y': []}}
    for _, row in df.iterrows():
        # Identify folder
        idx = row['id']
        split = 'train' if idx <= 4800 else 'validation' if idx <= 4900 else 'test'
        # Load EEGdata
        path = os.path.join(BASE_PATH, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
        df_eeg = pd.read_csv(path)
        # Extract correct segment
        n_samp = 2250 if row['task']=='MI' else 1750
        start = (row['trial']-1)*n_samp
        seg = df_eeg.iloc[int(start):int(start+n_samp)].reset_index(drop=True)
        proc = preprocess_trial(seg)
        data_dict[row['task']]['X'].append(proc.T)
        if has_label:
            data_dict[row['task']]['y'].append(row['label'])
    # Stack
    for t in data_dict:
        X = np.stack(data_dict[t]['X'])
        y = np.array(data_dict[t]['y']) if has_label else None
        data_dict[t]['X'] = X
        data_dict[t]['y'] = y
    return data_dict

# Execute processing and save
for fname, label_col in [('train.csv', True), ('validation.csv', True), ('test.csv', False)]:
    df_idx = load_index(fname, label_col)
    results = process_split(df_idx, label_col)
    for task, d in results.items():
        out_file = f"{os.path.splitext(fname)[0]}_{task}.npz"
        path = os.path.join(OUTPUT_DIR, out_file)
        if label_col:
            np.savez_compressed(path, X=d['X'], y=d['y'])
        else:
            np.savez_compressed(path, X=d['X'])
        print(f"Saved {out_file}: X={d['X'].shape}" + (f", y={d['y'].shape}" if d['y'] is not None else ''))
print('Preprocessing complete.')


Saved train_MI.npz: X=(2400, 8, 2250), y=(2400,)
Saved train_SSVEP.npz: X=(2400, 8, 1750), y=(2400,)
Saved validation_MI.npz: X=(50, 8, 2250), y=(50,)
Saved validation_SSVEP.npz: X=(50, 8, 1750), y=(50,)
Saved test_MI.npz: X=(50, 8, 2250)
Saved test_SSVEP.npz: X=(50, 8, 1750)
Preprocessing complete.


In [4]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- 0) Config ---
data_dir = './preprocessed'
output_dir = './cv_models'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ
n_folds = 5
batch_size = 64

# --- 1) Load & prepare data ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz = np.load(os.path.join(data_dir, 'validation_MI.npz'))

# Combine train and validation for CV
X_all = np.concatenate([train_npz['X'], val_npz['X']], axis=0)
y_all = np.concatenate([train_npz['y'], val_npz['y']], axis=0)

# Filter for EEG channels and transpose
X_all_raw = X_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)
y_all_bin = (y_all == 'Right').astype(int)  # Binarize labels

# --- 2) Model Builders (unchanged) ---
def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128, 5, activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", MacroF1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16, 32, 64, 128, 256]:
        x = layers.Conv1D(f, 3, activation="relu", padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128, 64, 32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", MacroF1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64, 3, activation="elu", padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", MacroF1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0, 2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:, :, C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:, :, C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16, 250, activation="relu", padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32, 50, activation="relu", padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3, b4])
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(eeg_in, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", MacroF1Score()])
    return m

# --- 3) Macro F1 Score Metric ---
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.classes = 2
        self.tp = [self.add_weight(name=f"tp_{i}", initializer="zeros") for i in range(self.classes)]
        self.fp = [self.add_weight(name=f"fp_{i}", initializer="zeros") for i in range(self.classes)]
        self.fn = [self.add_weight(name=f"fn_{i}", initializer="zeros") for i in range(self.classes)]

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=1)
        y_pred = tf.argmax(y_pred, axis=1)
        
        for i in range(self.classes):
            true_pos = tf.logical_and(tf.equal(y_true, i), tf.equal(y_pred, i))
            false_pos = tf.logical_and(tf.not_equal(y_true, i), tf.equal(y_pred, i))
            false_neg = tf.logical_and(tf.equal(y_true, i), tf.not_equal(y_pred, i))
            
            self.tp[i].assign_add(tf.reduce_sum(tf.cast(true_pos, tf.float32)))
            self.fp[i].assign_add(tf.reduce_sum(tf.cast(false_pos, tf.float32)))
            self.fn[i].assign_add(tf.reduce_sum(tf.cast(false_neg, tf.float32)))

    def result(self):
        f1_scores = []
        eps = tf.keras.backend.epsilon()
        for i in range(self.classes):
            precision = self.tp[i] / (self.tp[i] + self.fp[i] + eps)
            recall = self.tp[i] / (self.tp[i] + self.fn[i] + eps)
            f1 = 2 * precision * recall / (precision + recall + eps)
            f1_scores.append(f1)
        return tf.reduce_mean(f1_scores)

    def reset_states(self):
        for i in range(self.classes):
            self.tp[i].assign(0.0)
            self.fp[i].assign(0.0)
            self.fn[i].assign(0.0)

# --- 4) Cosine LR Schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 5) Callbacks ---
def get_callbacks(name):
    return [
        EarlyStopping(monitor="val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(output_dir, f"best_{name}.h5"),
                        monitor="val_f1_score", mode="max", save_best_only=True),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 6) Data Augmentation Generator ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

# --- 7) Apply CSP ---
def apply_csp(W, X):  # X: (n, T, C)
    return np.stack([W.dot(ep.T) for ep in X], axis=0).astype('float32')

# --- 8) Cross-Validation ---
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
models_to_run = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model5': build_model5,
    'model6': build_model6
}

# Store results
results = {name: [] for name in models_to_run}

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X_all_raw)):
    print(f"\n=== Fold {fold_idx+1}/{n_folds} ===")
    
    # Split data
    X_train_raw, X_test_raw = X_all_raw[train_idx], X_all_raw[test_idx]
    y_train_bin, y_test_bin = y_all_bin[train_idx], y_all_bin[test_idx]
    y_train_oh = keras.utils.to_categorical(y_train_bin, 2)
    y_test_oh = keras.utils.to_categorical(y_test_bin, 2)
    
    # Prepare CSP for model5 (using training data only)
    if 'model5' in models_to_run:
        csp = CSP(n_components=4, log=False, norm_trace=False)
        X_train_csp_input = X_train_raw.transpose(0, 2, 1).astype('float64')
        csp.fit(X_train_csp_input, y_train_bin)
        W = csp.filters_[:4]
        
        X_train_csp = apply_csp(W, X_train_raw).transpose(0, 2, 1)
        X_test_csp = apply_csp(W, X_test_raw).transpose(0, 2, 1)
    
    # Train each model
    for name, builder in models_to_run.items():
        print(f"\nTraining {name} (Fold {fold_idx+1})")
        
        # Select data
        if name == 'model5':
            X_train, X_test = X_train_csp, X_test_csp
        else:
            X_train, X_test = X_train_raw, X_test_raw
        
        # Create data generators
        train_gen = aug_gen(X_train, y_train_oh, seed=fold_idx, batch_size=batch_size)
        steps_per_epoch = len(X_train) // batch_size
        
        # Build and train model
        model = builder(X_train.shape[1:])
        model.fit(
            train_gen,
            steps_per_epoch=steps_per_epoch,
            epochs=200,
            validation_data=(X_test, y_test_oh),
            callbacks=get_callbacks(f"{name}_fold{fold_idx}"),
            verbose=2
        )
        
        # Evaluate with macro F1
        preds = np.argmax(model.predict(X_test), axis=1)
        f1 = f1_score(y_test_bin, preds, average='macro')  # Macro F1
        results[name].append(f1)
        print(f"{name} (Fold {fold_idx+1}) Macro F1: {f1:.4f}")
        print(classification_report(y_test_bin, preds, target_names=["Left", "Right"]))

# --- 9) Print CV Results ---
print("\n=== Cross-Validation Results ===")
for name, f1_scores in results.items():
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    print(f"{name}:")
    print(f"  Macro F1 Scores: {[f'{s:.4f}' for s in f1_scores]}")
    print(f"  Mean Macro F1: {mean_f1:.4f} ± {std_f1:.4f}\n")


=== Fold 1/5 ===
Computing rank from data with rank=None
    Using tolerance 3e+03 (2.2e-16 eps * 4 dim * 3.4e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

Training oldA (Fold 1)
Epoch 1/200
30/30 - 9s - 305ms/step - accuracy: 0.4995 - f1_score: 0.4977 - loss: 0.8474 - val_accuracy: 0.5000 - val_f1_score: 0.4250 - val_loss: 0.8936 - learning_rate: 0.0010
Epoch 2/200
30/30 - 1s - 30ms/step - accuracy: 0.4969 - f1_score: 0.4944 - loss: 0.7884 - val_accuracy: 0.5020 - val_f1_score: 0.5015 - val_loss: 0.7223 - learning_rate: 9.9994e-04
Epoch 3/200
30/30 - 1s - 28ms/step - accuracy: 0.4740 - f1_score: 0.4739 - loss: 0.7497 - val_accuracy: 0.4837 - val_f1_score: 0.4413 - val_loss: 0.7275 - learning_rate: 9.9969e-04
Epoch 4/200
30/30 - 1s - 28ms/step - accuracy: 0.5063 - f1_score: 0.506